In [5]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np

%load_ext cython
%load_ext snakeviz

<br></br>
<br></br>

# Performance considerations in scikit-learn
---

Good practices for efficient computation for users and developers

<br></br>
<br></br>
<div align="center">Jérémie du Boisberranger (INRIA)</div>

Problematic: Run machine learning algorithms on large datasets.

**Machine learning algorithms**
- Memory / Time complexity
- Implementation details

**Large datasets**
- Out of core
- Cache optimization
- Scalability

## Think about data types

Float32 vs float64
- 2x less memory
- faster computations (with vector instructions)
- most scikit-learn estimators support both
- numerical stability ensured by test suite

<div align='center'><img src="simd2_.png" width="70%"></div>

<div align='center'><img src="simd_.png" width="70%"></div>

An illustration with a simple matrix product

In [13]:
# dtype = float64
X = np.random.normal(size=(1000, 1000))
%timeit X @ X.T

14.7 ms ± 474 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
# dtype = float32
X = X.astype(np.float32)
%timeit X @ X.T

7.53 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


And using a scikit-learn estimator

In [4]:
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge

X, y = make_regression(n_samples=10000, n_features=1000)

In [5]:
%timeit Ridge().fit(X, y)

206 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
X = X.astype(np.float32)
y = y.astype(np.float32)

%timeit Ridge().fit(X, y)

112 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Type inference when reading data from file**

Example: loading data in a pandas DataFrame

data.csv is a csv file containing the following data:

a &nbsp; b &nbsp; c  
1 &nbsp; 1 &nbsp; 1.0  
2 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.0  
3 &nbsp; 3 &nbsp; 3.0 

In [7]:
import pandas as pd

df = pd.read_csv('data.csv')
print(df)

   a    b    c
0  1  1.0  1.0
1  2  NaN  2.0
2  3  3.0  3.0


In [8]:
df.dtypes

a      int64
b    float64
c    float64
dtype: object

**Intermediate computations on the data**

Example: scipy *pdist* converts to float64

In [9]:
from scipy.spatial.distance import pdist

X = np.random.normal(size=(1000, 1000))
X = X.astype(np.float32)

pdist(X).dtype

dtype('float64')

## Dealing with large intermediate results

*Example*: K-means  
For each point in X, find its closest center in C.

Large intermediate result: pairwise distances between X and C  
**n_samples x n_clusters**

Small final result: indices of closest center for each observation  
**n_samples**
  
- high memory consumption  
- slower execution (data do not fit in cpu cache)

=> Compute intermediate results by chunks
<div align='center'><img src="km_perf_vs_chunk_size_.png" width="70%"></div>
**trade-off memory vs speed**

**pairwise_distances_argmin** is the critical part of K-means.  


Let's see how we can configure scikit-learn to optimize it.

In [10]:
from sklearn.metrics import pairwise_distances_argmin

X = np.random.random_sample((100000, 100))
C = np.random.random_sample((1000, 100))

%timeit pairwise_distances_argmin(X, C)
%memit pairwise_distances_argmin(X, C)

1.14 s ± 34.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 1009.50 MiB, increment: 763.10 MiB


The **working_memory** configuration parameter allows to limit the size of the intermediate results.

In [11]:
from sklearn import config_context

with config_context(working_memory=0.1):
    %timeit pairwise_distances_argmin(X, C)
    %memit pairwise_distances_argmin(X, C)

2.38 s ± 13.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 248.25 MiB, increment: 1.35 MiB


In [12]:
with config_context(working_memory=1):
    %timeit pairwise_distances_argmin(X, C)
    %memit pairwise_distances_argmin(X, C)

942 ms ± 92.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 247.21 MiB, increment: 0.05 MiB


## Benefit from sparsity

Datasets with lot of zeros can be stored in sparse format.

Algorithms take account of sparse representation to reduce time and memory complexity.  
e.g. $O$(# elements) $\rightarrow$ $O$(# non zeros)

- Most scikit-learn estimators support sparse input.
- Preprocessing transformers return sparse output when appropriate **CountVectorizer**, **OneHotEncoder**, ...

In [13]:
from sklearn.preprocessing import OneHotEncoder

X = np.random.randint(100, size=(10000, 10))
y = np.random.randint(2, size=10000)

Xt = OneHotEncoder(categories='auto').fit_transform(X)
Xt

<10000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [14]:
# density
Xt.nnz / np.prod(Xt.shape)

0.01

In [3]:
from scipy.sparse import csr_matrix

def make_data(n_samples=100, n_features=100, density=1, random_state=0):
    rng = np.random.RandomState(random_state)
    X = rng.random_sample((n_samples, n_features))
    X[X < (1 - density)] = 0

    y = rng.randint(2, size=n_samples)
    
    return X, csr_matrix(X), y

When the data is sufficiently sparse, the speed-up can be significant

In [1]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=10, max_depth=5,
                           max_features='log2')

In [6]:
X_dense, X_sparse, y = make_data(n_samples=10000, n_features=1000,
                                 density=0.01)

%timeit rf.fit(X_dense, y)
%timeit rf.fit(X_sparse, y)

116 ms ± 5.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
17.3 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


However when it's not sparse enough, the performance can be degraded.

In [18]:
X_dense, X_sparse, y = make_data(n_samples=10000, n_features=1000,
                                 density=0.5)

%timeit rf.fit(X_dense, y)
%timeit rf.fit(X_sparse, y)

198 ms ± 976 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
251 ms ± 2.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Avoid unnecessary copy

Some data attributes can cause a copy:
- memory layout (C / Fortran)
- dtype (float32 / float64)
- sparse format (CSR / CSC)

In [22]:
from sklearn.svm import LinearSVR

X = np.random.random_sample((100000, 100))
y = np.random.random_sample(100000)

svr = LinearSVR(max_iter=20, tol=1e-16)
%memit svr.fit(X, y)

peak memory: 525.66 MiB, increment: 113.76 MiB


In [23]:
X = np.asfortranarray(X)
%memit svr.fit(X, y)

peak memory: 602.00 MiB, increment: 190.09 MiB


In [24]:
X.nbytes / 2**20

76.2939453125

## Implement critical part in compiled language

- C, C++, Rust
- Cython <- scikit-learn
- Pythran
- Numba

Can be worth even when easily doable with numpy

In [14]:
X = np.random.random_sample((1000, 1000))
Y = np.random.random_sample((1000, 1000))
Z = np.random.random_sample((1000, 1000))

%timeit X + 2*Y + 3*Z

7.29 ms ± 685 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%cython
#cython: boundscheck=False, wraparound=False
import numpy as np

def cython_sum(double[:, ::1] X, double[:, ::1] Y, double[:, ::1] Z):
    cdef:
        Py_ssize_t m = X.shape[0]
        Py_ssize_t n = X.shape[1]
        Py_ssize_t i, j
        double[:, ::1] res = np.empty((m, n))
        
    for i in range(m):
        for j in range(n):
            res[i, j] = X[i, j] + 2 * Y[i, j] + 3 * Z[i, j]
    
    return np.asarray(res)

In [16]:
%timeit cython_sum(X, Y, Z)

1.94 ms ± 68.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%cython
#cython: boundscheck=False, wraparound=False
import numpy as np

def cython_sum(double[:, ::1] X, double[:, ::1] Y, double[:, ::1] Z):
    cdef:
        Py_ssize_t m = X.shape[0]
        Py_ssize_t n = X.shape[1]
        Py_ssize_t i, j
        double[:, ::1] res = np.empty((m, n))
        
    for i in range(m):
        for j in range(n):
            res[i, j] = X[i, j] + 2 * Y[i, j] + 3 * Z[i, j]
    
    return np.asarray(res)

## Parallelism

multi-processing:
- data copy
- inter-process communication
- in scikit-learn: joblib (loky backend)
    
multi-threading:
- data shared between threads
- thread safety questions
- in sklearn: joblib (threading backend), but GIL. OpenMP in cython.

In scikit-learn: controlled by the **n_jobs** parameter

In [28]:
from sklearn.metrics import pairwise_distances

X = np.random.normal(size=(1000, 1000))

In [29]:
%timeit pairwise_distances(X, metric='manhattan', n_jobs=1)
%timeit pairwise_distances(X, metric='manhattan', n_jobs=2)
%timeit pairwise_distances(X, metric='manhattan', n_jobs=4)

800 ms ± 84.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
445 ms ± 50 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
345 ms ± 73.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


However greater **n_jobs** does not always mean better performances

In [30]:
%timeit pairwise_distances(X, metric='euclidean', n_jobs=1)
%timeit pairwise_distances(X, metric='euclidean', n_jobs=2)
%timeit pairwise_distances(X, metric='euclidean', n_jobs=4)

21.8 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
101 ms ± 38.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
115 ms ± 2.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


- Thread or process management overhead
- Already multi-threaded by BLAS (Basic Linear Algebra Subroutines)

**Risk of oversubscription**

Nested parallelism common in scikit-learn
- Outer parallelism with joblib, controlled by **n_jobs**  
  mostly multi-processing
- Inner parallelism with BLAS (OpenBLAS / MKL), unrestricted  
  multi-threading

*Example*:  
0uter parallel loop spawns n (# cpu) processes  
Inner parallel loop spawns n (# cpu) threads  
=> n² processes/threads

An illustration with a search for best parameters of an estimator which makes lot of BLAS calls. (on a machine with 44 cpus)


First: our baseline with **n_jobs=1**

In [ ]:
In [2]: from sklearn.linear_model import LogisticRegression 
   ...: from sklearn.model_selection import GridSearchCV
   ...:  
   ...: X = np.random.random_sample((100000, 100)) 
   ...: y = np.random.randint(2, size=100000) 
   ...:
   ...: lr = LogisticRegression(solver='lbfgs') 
   ...: param_grid = {'C': np.logspace(-1, 3, num=10)} 
   ...:  
   ...: gs = GridSearchCV(lr, param_grid=param_grid,
   ...:                   cv=5, n_jobs=1) 
   ...:
   ...: %timeit gs.fit(X,y)                                                                                                             
27.9 s ± 369 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

**GrisSearchCV** is embarrassingly parallel.  
Let's put **n_jobs=44** to benefit from our 44 cpus.

In [ ]:
In [3]: gs = GridSearchCV(lr, param_grid=param_grid,
   ...:                   cv=5, n_jobs=44) 
   ...:  
   ...: %timeit gs.fit(X,y)                                                                                                
29.8 s ± 302 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

No improvement ! Even slightly worse...

<div align='center'><img src="gs44_ob44_.png" width="70%"></div>

We can restrict the allowed number of threads for the BLAS libraries.  
=> Set the **OPENBLAS_NUM_THREADS** or **MKL_NUM_THREADS** environment variable.  
(Before imports)

In [ ]:
In [2]: gs = GridSearchCV(lr, param_grid=param_grid,
   ...:                   cv=5, n_jobs=44) 
   ...:  
   ...: %timeit gs.fit(X,y)  
3.03 s ± 49.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

<div align='center'><img src="gs44_ob1_.png" width="70%"></div>

In the future joblib should automatically prevent oversubscription.  
=> Limit number of threads BLAS libraries can use in each sub-process.

For multi-threaded parallelism (e.g. with OpenMP in cython)  
=> Ongoing work in threadpoolctl, https://github.com/joblib/threadpoolctl.  
(inspired by SMP, https://github.com/IntelPython/smp, from Anton Malakhov)

## Take a step back: profile first!


Importance of profiling: find what needs an optimization the most.  
**Optimizing is trading maintainability for performance.**
 
- Only optimize the critical part.  
  Optimized code is often less readable.
  
  
- CPU bound vs memory bound.  
  Don't buy brand new hardware with faster memory access to multiply matrices.
  
  
- Ratio between sequential part and parallel part.  
  More cpus can only speed up the parallel part.

In [31]:
from sklearn.cluster import KMeans

X = np.random.random_sample((10000, 100))

kmeans = KMeans(n_clusters=100, n_init=1, algorithm='full')

%snakeviz -t kmeans.fit(X)

 
*** Profile stats marshalled to file '/tmp/tmpix05higg'. 
Opening SnakeViz in a new tab...


<img src="prof_.png" width="90%">

<br></br>
<br></br>

# Thank you for your attention

<br></br>
<br></br>
<div align='center'>https://github.com/jeremiedbb/performance-scikit-learn</div>